In [1]:
%load_ext lab_black

In [2]:
import json
import logging
import math
import os
import random
import shutil
import sys
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm import tqdm

import networks
import utils
from error_list import error_list

---

## 하이퍼 파라미터

In [3]:
POSE_MODEL = "HRNet-W48"
DET_PRETRAINED = ""
RESULT_DIR = Path("results/hrnet+det")

LR = 1e-4  # transfer learning이니깐 좀 작게 주는게 좋을 것 같아서 1e-4
BATCH_SIZE = 40
START_EPOCH = 1
SAM = True
FOLDS = [1, 2, 3, 4, 5]
PADDING = 30

n = datetime.now()
UID = f"{n.year:04d}{n.month:02d}{n.day:02d}-{n.hour:02d}{n.minute:02d}{n.second:02d}"
SEED = 20210309

In [4]:
torch.set_grad_enabled(False)

---

## 데이터셋 준비

In [22]:
test_imgs = np.array(sorted(list(Path("data/test_imgs_effdet/").glob("*.jpg"))))

with open("data/test_imgs_effdet/data.json", "r") as f:
    data = json.load(f)
    offsets = data["offset"]
    ratios = data["ratio"]

In [24]:
class KeypointDataset(Dataset):
    def __init__(self, files, offsets, ratios, augmentation=False):
        super().__init__()
        self.files = files
        self.offsets = offsets
        self.ratios = ratios

        T = []
        if augmentation:
            T.append(A.ImageCompression())
            T.append(A.Cutout())
            T_ = []
            T_.append(A.RandomBrightnessContrast())
            T_.append(A.RandomGamma())
            T_.append(A.RandomBrightness())
            T_.append(A.RandomContrast())
            T.append(A.OneOf(T_))
            T.append(A.GaussNoise())
            T.append(A.Blur())
        T.append(A.Normalize())
        T.append(ToTensorV2())

        self.transform = A.Compose(transforms=T)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = str(self.files[idx])
        offset = torch.tensor(self.offsets[idx], dtype=torch.float32)
        ratio = torch.tensor(self.ratios[idx], dtype=torch.float32)

        image = imageio.imread(file)
        a = self.transform(image=image)
        image = a["image"]

        return file, image, offset, ratio

---

## Validation

In [17]:
class TrainInputBean:
    def __init__(self):
        # HRNet 생성
        if POSE_MODEL == "HRNet-W32":
            width = 32
        elif POSE_MODEL == "HRNet-W48":
            width = 48
        else:
            raise NotImplementedError()

        self.pose_model = networks.PoseHighResolutionNet(width)
        self.pose_model.load_state_dict(torch.load(f"networks/models/pose_hrnet_w{width}_384x288.pth"))

        final_layer = nn.Conv2d(width, 24, 1)
        with torch.no_grad():
            final_layer.weight[:17] = self.pose_model.final_layer.weight
            final_layer.bias[:17] = self.pose_model.final_layer.bias
            self.pose_model.final_layer = final_layer
        self.pose_model.cuda()

    def load(self, path):
        print("Load pretrained", path)
        ckpt = torch.load(path)
        self.pose_model.load_state_dict(ckpt["model"])

---

## 모델 생성

In [27]:
ds_test = KeypointDataset(test_imgs, offsets, ratios, augmentation=False)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

In [28]:
B = TrainInputBean()
B.load("results/hrnet+det/ckpt-20210320-114245_1.pth")
B.pose_model.eval()
pass

Load pretrained results/hrnet+det/ckpt-20210320-114245_1.pth


---

## submission 만들기

In [29]:
for files, imgs, offsets, ratios in dl_test:
    break

In [32]:
ratios.shape

torch.Size([40, 2])

In [43]:
file_names = []
result = []
with tqdm(total=len(dl_test.dataset), ncols=100, file=sys.stdout) as t:
    for files, imgs, offsets, ratios in dl_test:
        pred_heatmaps = B.pose_model(imgs.cuda()).cpu()
        for file, pred_heatmap, offset, ratio in zip(files, pred_heatmaps, offsets, ratios):
            file = Path(file)
            file_names.append(file.name)
            t.set_postfix_str(file.name)

            keypoint = utils.heatmaps2keypoints(pred_heatmap)
            keypoint = keypoint * 4 / ratio.view(1, 2) + offset.view(1, 2)

            # effdet의 ROI
            keypoint[:, 1] += 28
            keypoint[:, 0] *= 1920 / 1024
            keypoint[:, 1] *= 1024 / 512
            result.append(keypoint)
            t.update()

100%|████████████████████████████| 1600/1600 [00:13<00:00, 122.18it/s, 786-3-5-41-Z94_E-0000031.jpg]


In [45]:
file_names = np.array(file_names)

In [47]:
result = torch.stack(result)

In [67]:
submit_data = np.concatenate([np.expand_dims(file_names, 1), result.flatten(1).numpy()], 1)

In [70]:
sample_submission = pd.read_csv("data/ori/sample_submission.csv")

In [71]:
submit_df = pd.DataFrame(submit_data, columns=sample_submission.columns)

In [72]:
submit_df

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,649-2-4-32-Z148_A-0000001.jpg,1097.2135,545.9375,1097.2135,565.8541,1119.6354,560.875,1067.3176,620.625,1157.0052,...,977.6302,316.89584,977.6302,555.8959,925.3125,521.0416,678.6719,725.1875,820.6771,680.375
1,649-2-4-32-Z148_A-0000003.jpg,1107.8126,544.875,1100.2865,564.7084,1122.8645,559.75,1070.1824,619.25,1160.4948,...,1017.50006,401.0833,979.8698,554.7916,912.13544,530.0,678.8281,723.375,821.8229,678.75
2,649-2-4-32-Z148_A-0000005.jpg,1096.0156,546.1666,1103.4375,561.2916,1118.2812,561.2916,1066.3282,621.7916,1155.3907,...,1014.375,314.25,955.0,566.3334,888.2031,551.2084,680.3906,722.625,821.40625,682.2916
3,649-2-4-32-Z148_A-0000007.jpg,1111.7188,562.2292,1084.375,571.5417,1148.177,549.8125,1047.9167,611.8959,1148.177,...,1257.5521,608.7917,1084.375,667.7709,902.0833,531.1875,674.21875,723.6459,820.05206,680.1875
4,649-2-4-32-Z148_A-0000009.jpg,1097.552,543.9375,1097.552,563.8541,1119.8959,314.89584,1067.7604,623.6041,1157.1355,...,1105.0,334.8125,956.0417,558.875,896.4583,548.9166,680.46875,723.1875,821.9792,678.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,786-3-5-41-Z94_E-0000023.jpg,691.0416,601.4583,684.6875,593.25,684.6875,589.1458,633.8541,580.9375,703.74994,...,868.9584,736.8959,640.2084,687.6459,697.3959,720.4792,938.8542,913.375,983.3334,872.3334
1596,786-3-5-41-Z94_E-0000025.jpg,931.6667,622.75,911.4844,611.5,911.4844,611.5,915.5208,600.25,935.7031,...,1056.7969,750.25,963.95825,776.5,846.901,720.25,947.8125,900.25,947.8125,900.25
1597,786-3-5-41-Z94_E-0000027.jpg,687.3958,588.1667,748.6979,572.0833,709.6875,596.2083,698.5416,584.1458,726.40625,...,849.0104,724.875,787.7083,636.4167,720.8334,636.4167,938.177,909.8334,960.46875,889.7292
1598,786-3-5-41-Z94_E-0000029.jpg,847.7344,623.625,875.9896,563.9375,884.0625,766.875,847.7344,587.8125,855.80725,...,968.82806,731.0625,815.4427,643.5208,811.40625,830.5417,997.0833,906.1459,960.7552,902.1667


In [73]:
submit_df.describe()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
count,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600,...,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600
unique,1600,1322,1350,1367,1373,1350,1356,1342,1333,1375,...,1382,1414,1337,1360,1308,1340,1340,1337,1316,1287
top,754-3-5-38-Z94_A-0000029.jpg,941.25,554.0,1171.875,502.0,1163.4375,494.9375,1049.349,602.0,1171.875,...,849.84375,304.25,942.1875,565.5625,942.1875,605.5,741.5625,918.6875,640.3125,826.0
freq,1,11,9,7,9,7,6,6,7,9,...,6,8,9,8,9,7,8,8,8,8


In [74]:
submit_df.to_csv("results/hrnet+det/submit-20210320-114245_1.csv", index=False)